### Libraries

In [1]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qclib.gates.ldmcu import Ldmcu
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import state_fidelity, Statevector, DensityMatrix, Operator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import matplotlib.pyplot as plt

In [2]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def transpose_conjugate(operator):
        """Returns a transposed conjugate matrix of the operator"""
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        """Returns the base control size of the pyramid"""
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [27]:
def build_circuit_experiment(extra_qubits=0 , error=0.1, approximated=True, op_matrix=None):
    if(op_matrix==None):
        op_matrix = Operator.from_label('X').data
    n_base = Utilities.pyramid_size(op_matrix, error)
    print("N Base ", n_base)
    controls = QuantumRegister(n_base + extra_qubits, 'controls')
    target = QuantumRegister(1, 'target')
    circ = QuantumCircuit(controls, target)
    circ.x(controls)

    circ.unitary(op_matrix, target)
    if approximated:
        MCU.mcu(circ, Utilities.transpose_conjugate(op_matrix), controls, target, error)
    else:
        Ldmcu.ldmcu(circ, Utilities.transpose_conjugate(op_matrix), controls, target)
    circ.measure_all()
    return circ


In [4]:
print(build_circuit_experiment(approximated=True))

               ┌───┐   ┌──────────────┐ ░ ┌─┐                  
controls_0: ───┤ X ├───┤0             ├─░─┤M├──────────────────
               ├───┤   │              │ ░ └╥┘┌─┐               
controls_1: ───┤ X ├───┤1             ├─░──╫─┤M├───────────────
               ├───┤   │              │ ░  ║ └╥┘┌─┐            
controls_2: ───┤ X ├───┤2             ├─░──╫──╫─┤M├────────────
               ├───┤   │              │ ░  ║  ║ └╥┘┌─┐         
controls_3: ───┤ X ├───┤3 Ldmcuapprox ├─░──╫──╫──╫─┤M├─────────
               ├───┤   │              │ ░  ║  ║  ║ └╥┘┌─┐      
controls_4: ───┤ X ├───┤4             ├─░──╫──╫──╫──╫─┤M├──────
               ├───┤   │              │ ░  ║  ║  ║  ║ └╥┘┌─┐   
controls_5: ───┤ X ├───┤5             ├─░──╫──╫──╫──╫──╫─┤M├───
            ┌──┴───┴──┐│              │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
    target: ┤ Unitary ├┤6             ├─░──╫──╫──╫──╫──╫──╫─┤M├
            └─────────┘└──────────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
    meas: 7/════════════════════════════

In [5]:
len(build_circuit_experiment(approximated=True).qubits)

7

In [5]:
def create_noise_model():
    #noise_model = NoiseModel.from_backend(backend)
    # Create an empty noise model
    noise_model = NoiseModel()

    # Add depolarizing error to all single qubit u1, u2, u3 gates
    error_1qubit = depolarizing_error(0.0001, 1)
    error_2qubit = depolarizing_error(0.001, 2)
    noise_model.add_all_qubit_quantum_error(error_1qubit, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(error_2qubit, ['cx'])
    # Measurement miss-assignement probabilities (No Effect)
    #p0given1 = 0.5
    #p1given0 = 0.5
    #readout = ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])
    #noise_model.add_all_qubit_readout_error(readout)
    return noise_model

def get_fake_backend():
    from qiskit.providers.fake_provider import Fake27QPulseV1
    from qiskit_aer.noise import NoiseModel
    backend = Fake27QPulseV1()
    noise_model = NoiseModel.from_backend(backend)
    return noise_model



In [20]:
def density_matrix(circuit, fake_backend=False):
    
    if(fake_backend==False): 
        noise_model = create_noise_model()
    else:
        noise_model = get_fake_backend()
    simulator = AerSimulator(noise_model=noise_model, method='density_matrix')
    transpiled_circuit = transpile(circuit, simulator, basis_gates= noise_model.basis_gates)
    transpiled_circuit.save_density_matrix()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    d_m = result.data(0)
    
    return d_m

def experiment_counts(circuit, fake_backend=False, shots=1024):
    
    if(fake_backend==False): 
        noise_model = create_noise_model()
    else:
        noise_model = get_fake_backend()
    simulator = AerSimulator(noise_model=noise_model)
    transpiled_circuit = transpile(circuit, simulator, basis_gates= noise_model.basis_gates)
    transpiled_circuit.save_density_matrix()
    
    job = simulator.run(transpiled_circuit, shots = shots)
    result = job.result()
    counts = result.get_counts(0)
    
    return counts

In [21]:
def run_experiment(error=0.1, n_extra=1, rounds=10, op_matrix=None, IBM_washington = True, shots=1024):

    approx_counts = np.zeros(rounds)
    exact_counts = np.zeros(rounds)

    print("Error ", error)
    print("N_extra ", n_extra)

    for i in range(rounds):
        approx_circuit = build_circuit_experiment(n_extra, error)
        real_circuit = build_circuit_experiment(n_extra, error, False)
        correct_bit='0'
        for _ in range(len(approx_circuit.qubits)-1):
            correct_bit+='1'
        approx_counts[i] = experiment_counts(approx_circuit, fake_backend=IBM_washington, shots=1024)[correct_bit]/1024
        exact_counts[i] = experiment_counts(real_circuit, fake_backend=IBM_washington, shots=1024)[correct_bit]/1024
        #total_approx = sum(dict(approx_counts).values())
        #print(total_approx)
        #print(sum(dict(exact_counts).values()))
        
        
    print("Exact ", exact_counts)
    print("Mean ", exact_counts.mean(), "std ", exact_counts.std())
    print("Approx ", approx_counts)
    print("Mean ", approx_counts.mean(), "std ", approx_counts.std())



### Experimentos simulando IBM Washington

In [27]:
run_experiment(rounds=30)

Error  0.1
N_extra  1
Exact  [0.65625    0.69824219 0.71972656 0.671875   0.67480469 0.67675781
 0.6796875  0.68945312 0.68554688 0.68066406 0.67578125 0.67089844
 0.71386719 0.6640625  0.69726562 0.68847656 0.68457031 0.68847656
 0.68554688 0.68164062 0.68359375 0.68066406 0.67089844 0.69824219
 0.67578125 0.66699219 0.69628906 0.67578125 0.69238281 0.68847656]
Mean  0.6837565104166666 std  0.01341386791915836
Approx  [0.74902344 0.72363281 0.72265625 0.71972656 0.74804688 0.734375
 0.71972656 0.7421875  0.69921875 0.73046875 0.69238281 0.69238281
 0.72753906 0.71484375 0.73730469 0.73925781 0.7265625  0.72558594
 0.74316406 0.69335938 0.72460938 0.71582031 0.74902344 0.73144531
 0.73144531 0.7421875  0.68554688 0.70996094 0.73046875 0.7109375 ]
Mean  0.7237630208333333 std  0.01749243678796113


In [22]:
run_experiment(rounds=10, shots=8.192)

Error  0.1
N_extra  1
Exact  [0.70410156 0.69433594 0.68847656 0.68164062 0.68359375 0.67578125
 0.66210938 0.69140625 0.70117188 0.70898438]
Mean  0.68916015625 std  0.013383182482663002
Approx  [0.75       0.69921875 0.72949219 0.74121094 0.70703125 0.71484375
 0.703125   0.72558594 0.72070312 0.73242188]
Mean  0.72236328125 std  0.01575295508379524


In [29]:
run_experiment(rounds=30, n_extra=2)

Error  0.1
N_extra  2
Exact  [0.59277344 0.59570312 0.54882812 0.59375    0.609375   0.59960938
 0.59375    0.61328125 0.58886719 0.5859375  0.59863281 0.6015625
 0.58398438 0.59472656 0.58300781 0.58203125 0.58203125 0.60644531
 0.56835938 0.61328125 0.59960938 0.59960938 0.59667969 0.58984375
 0.58007812 0.58886719 0.60058594 0.57324219 0.58300781 0.60839844]
Mean  0.5918619791666667 std  0.013564340242935829
Approx  [0.62988281 0.61621094 0.6171875  0.625      0.60058594 0.61816406
 0.61425781 0.63867188 0.61523438 0.63769531 0.60546875 0.62890625
 0.61230469 0.609375   0.62207031 0.62304688 0.63085938 0.62988281
 0.59765625 0.61328125 0.60351562 0.64257812 0.61816406 0.62109375
 0.61621094 0.6015625  0.62597656 0.62011719 0.60351562 0.62695312]
Mean  0.61884765625 std  0.011453014642132598


In [31]:
run_experiment(rounds=10, n_extra=3)

Error  0.1
N_extra  3
Exact  [0.55371094 0.53222656 0.51367188 0.55566406 0.54199219 0.56347656
 0.52539062 0.54394531 0.54785156 0.50683594]
Mean  0.5384765625 std  0.017633376707854163
Approx  [0.55371094 0.56933594 0.52539062 0.56640625 0.56347656 0.56542969
 0.53613281 0.53515625 0.5546875  0.56542969]
Mean  0.553515625 std  0.014937256878905385


In [28]:
run_experiment(rounds=30, error=0.2)

Error  0.2
N_extra  1
Exact  [0.72167969 0.72070312 0.72558594 0.71289062 0.70605469 0.69335938
 0.71679688 0.73632812 0.72460938 0.71191406 0.70898438 0.72558594
 0.71972656 0.7265625  0.70703125 0.70703125 0.70214844 0.70703125
 0.71386719 0.71777344 0.73144531 0.71972656 0.67773438 0.70214844
 0.71875    0.71484375 0.70898438 0.71679688 0.70117188 0.71679688]
Mean  0.7138020833333333 std  0.011579394212747117
Approx  [0.74511719 0.73730469 0.7578125  0.73046875 0.75683594 0.78125
 0.76757812 0.73144531 0.7578125  0.74609375 0.74023438 0.75683594
 0.73632812 0.76074219 0.75       0.76367188 0.75       0.74804688
 0.76757812 0.77636719 0.74609375 0.76367188 0.765625   0.75195312
 0.765625   0.74023438 0.7734375  0.75585938 0.77246094 0.73535156]
Mean  0.75439453125 std  0.013580306263889215


In [30]:
run_experiment(rounds=30, error=0.2, n_extra=2)

Error  0.2
N_extra  2
Exact  [0.69238281 0.70605469 0.7109375  0.69824219 0.69433594 0.69824219
 0.69238281 0.70214844 0.69042969 0.69726562 0.67675781 0.65917969
 0.68261719 0.66210938 0.67871094 0.68652344 0.67773438 0.67578125
 0.68164062 0.68457031 0.69433594 0.70703125 0.6796875  0.68359375
 0.65722656 0.67871094 0.6640625  0.6875     0.68847656 0.703125  ]
Mean  0.6863932291666667 std  0.013823869892245913
Approx  [0.73632812 0.73925781 0.74316406 0.7421875  0.74804688 0.76074219
 0.76855469 0.73828125 0.74902344 0.77246094 0.75976562 0.76660156
 0.74414062 0.74804688 0.74609375 0.76367188 0.70703125 0.74707031
 0.76367188 0.73925781 0.76464844 0.74316406 0.76074219 0.77441406
 0.75488281 0.75585938 0.73339844 0.73144531 0.72167969 0.72753906]
Mean  0.7483723958333334 std  0.01545178511703264


Peguei esses dados abaixo

In [23]:
run_experiment(rounds=50, op_matrix=Operator.from_label('Y').data, shots=8192)

Error  0.1
N_extra  1
Exact  [0.66796875 0.66699219 0.69921875 0.6875     0.69433594 0.69824219
 0.67480469 0.70214844 0.70214844 0.68652344 0.66992188 0.69335938
 0.67382812 0.703125   0.68066406 0.69433594 0.6640625  0.71191406
 0.67382812 0.70703125 0.69628906 0.69140625 0.67089844 0.70898438
 0.71875    0.67285156 0.66308594 0.68164062 0.6953125  0.68457031
 0.69140625 0.671875   0.67480469 0.68261719 0.69042969 0.70117188
 0.67382812 0.68652344 0.70410156 0.67578125 0.70117188 0.70410156
 0.66503906 0.66699219 0.68457031 0.70019531 0.71191406 0.66992188
 0.68359375 0.68164062]
Mean  0.6871484375 std  0.014657236940384176
Approx  [0.74609375 0.74511719 0.74902344 0.75292969 0.734375   0.74121094
 0.72460938 0.73925781 0.72363281 0.75878906 0.71972656 0.73632812
 0.74023438 0.74511719 0.70898438 0.71191406 0.71972656 0.73730469
 0.72753906 0.74316406 0.72167969 0.71875    0.72167969 0.69824219
 0.75097656 0.71875    0.73144531 0.74414062 0.75195312 0.72265625
 0.71679688 0.76074219 

In [26]:
Utilities.pyramid_size(operator=Operator.from_label('Y').data, error = 0.1)

6

In [28]:
run_experiment(rounds=1, op_matrix=Operator.from_label('Y').data)

Error  0.1
N_extra  1
N Base  6
N Base  6
Exact  [0.66601562]
Mean  0.666015625 std  0.0
Approx  [0.734375]
Mean  0.734375 std  0.0


In [9]:
run_experiment(rounds=30, op_matrix=Operator.from_label('S').data)

Error  0.1
N_extra  1
Exact  [0.69335938 0.68652344 0.65917969 0.6953125  0.66796875 0.66601562
 0.70605469 0.71484375 0.70117188 0.71679688 0.68945312 0.69824219
 0.66308594 0.68359375 0.71582031 0.7109375  0.66699219 0.67285156
 0.70019531 0.69433594 0.69238281 0.6640625  0.66503906 0.67773438
 0.69433594 0.703125   0.69628906 0.69628906 0.65625    0.68945312]
Mean  0.6879231770833333 std  0.017698307565163267
Approx  [0.75683594 0.71289062 0.72949219 0.7421875  0.71484375 0.73242188
 0.73339844 0.734375   0.75488281 0.70117188 0.71777344 0.74023438
 0.74023438 0.734375   0.7265625  0.72070312 0.73242188 0.73242188
 0.73632812 0.72558594 0.75097656 0.74804688 0.72851562 0.73144531
 0.73632812 0.7265625  0.71386719 0.71191406 0.73242188 0.69433594]
Mean  0.72978515625 std  0.014272854330074945


Experimento antigo para comparação

In [45]:
n_extra = [1]
M = 10
fidelity_exact = np.zeros(M)
fidelity_approx = np.zeros(M)

error = 0.1
matrix = 'x'
fake_backend =  True

print("Error ", error)
for i in range(len(n_extra)):
    ne = n_extra[i]
    print("N_extra ", ne)
    approx_circuit = build_circuit_experiment(ne, error)
    real_circuit = build_circuit_experiment(ne, error, False)
    approx_counts = experiment_counts(approx_circuit, fake_backend=True)
    exact_counts = experiment_counts(real_circuit, fake_backend=True)
    #total_approx = sum(dict(approx_counts).values())
    #print(total_approx)
    #print(sum(dict(exact_counts).values()))
    s='0'
    for a in range(len(approx_circuit.qubits)-1):
        s+='1'
    
    print("Approx acc ", approx_counts[s]/1024)
    print("Exact acc ", exact_counts[s]/1024)



Error  0.1
N_extra  0
Approx acc  0.703125
Exact acc  0.7158203125
N_extra  1
Approx acc  0.7451171875
Exact acc  0.6845703125
N_extra  2
Approx acc  0.6533203125
Exact acc  0.5771484375
N_extra  3
Approx acc  0.55078125
Exact acc  0.5166015625
